# **ElasticNet Regression**

ElasticNet Regression is a linear regression model that combines the properties of both Lasso and Ridge regression. 

It is particularly useful when dealing with datasets that have a large number of features, some of which may be correlated. 

The ElasticNet model applies both L1 (Lasso) and L2 (Ridge) regularization to the loss function, allowing it to perform variable selection and shrinkage simultaneously.

This makes it effective for high-dimensional datasets where traditional linear regression may overfit or fail to generalize well.

It is commonly used in machine learning tasks where feature selection and regularization are important for improving model performance and interpretability.

In [120]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

In [121]:
import seaborn as sns
df = sns.load_dataset("diamonds")
print(df.head())

   carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2   0.23     Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3   0.29  Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4   0.31     Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [123]:
print("Memory usage before (MB):", df.memory_usage(deep=True).sum() / 1024**2)

Memory usage before (MB): 3.036884307861328


In [124]:
# Downcast float columns
for col in df.select_dtypes(include=['float64', 'float32']).columns:
    if df[col].isna().any():
        print(f"Warning: Column {col} contains NaN values")
    df[col] = pd.to_numeric(df[col], downcast='float', errors='coerce')

# Downcast integer columns
for col in df.select_dtypes(include=['int64', 'int32', 'int16']).columns:
    df[col] = pd.to_numeric(df[col], downcast='integer')

# Convert object columns to category (already done in your output)
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float32 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float32 
 5   table    53940 non-null  float32 
 6   price    53940 non-null  int16   
 7   x        53940 non-null  float32 
 8   y        53940 non-null  float32 
 9   z        53940 non-null  float32 
dtypes: category(3), float32(6), int16(1)
memory usage: 1.5 MB


In [126]:
# Downcasting code here
print("Memory usage after (MB):", df.memory_usage(deep=True).sum() / 1024**2)

Memory usage after (MB): 1.4936485290527344


In [127]:
X = df.drop(columns=['price'], axis=1)
y = df['price']

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [129]:
print("Number of samples in df['cut']:", len(df['cut']))
print("Number of samples in y_train:", len(y_train))

Number of samples in df['cut']: 53940
Number of samples in y_train: 43152


In [130]:
print("X_train dtypes:\n", X_train.dtypes)
print("y_train dtype:", y_train.dtype)

X_train dtypes:
 carat       float32
cut        category
color      category
clarity    category
depth       float32
table       float32
x           float32
y           float32
z           float32
dtype: object
y_train dtype: int16


In [131]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [132]:
# Downcast numeric columns to float32
numeric_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']
X_train[numeric_cols] = X_train[numeric_cols].astype('float32')
X_test[numeric_cols] = X_test[numeric_cols].astype('float32')

In [133]:
from sklearn.preprocessing import TargetEncoder

target_encoder = TargetEncoder(target_type='continuous')
X_train_encoded = target_encoder.fit_transform(X_train, y_train)
X_test_encoded = target_encoder.transform(X_test)

df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.500000,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.799999,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.900002,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.400002,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.299999,58.0,335,4.34,4.35,2.75


In [134]:
en = ElasticNet()
en.fit(X_train_encoded, y_train)

ElasticNet()

In [135]:
y_pred = en.predict(X_test_encoded)
display(display(pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})))

,Actual,Predicted
1388,559.0,1539.180703
50052,2201.0,2396.888213
41645,1238.0,2016.806585
42377,1304.0,1837.779367
17244,6901.0,11355.112633
...,...,...
44081,1554.0,2032.783482
23713,633.0,954.145474
31375,761.0,1548.998189
21772,9836.0,7724.391134


None

In [136]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

print("Accuracy 1:", en.score(X_train_encoded, y_train))
print("Mean Squared Error 1:", mean_squared_error(y_test, y_pred))
print("R^2 Score 1:", r2_score(y_test, y_pred))
print("Mean Absolute Error 1:", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error 1:", root_mean_squared_error(y_test, y_pred))

Accuracy 1: 0.9079902451361378
Mean Squared Error 1: 1499314.3971436524
R^2 Score 1: 0.9056846425720626
Mean Absolute Error 1: 755.2903579452123
Root Mean Squared Error 1: 1224.4649432072983


In [141]:
# L1 and L2 regularization parameters for ElasticNet
# L1 ratio is the mix ratio between L1 and L2 regularization
# It can take values between 0 and 1, where 0 is L2 (Ridge) and 1 is L1 (Lasso)
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 0.5, 1.0],
    'l1_ratio': [0.1, 0.5, 0.9, 1.0]
}

In [143]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=ElasticNet(), param_grid=param_grid, cv=5, n_jobs=2, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train_encoded, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=2,
             param_grid={'alpha': [0.001, 0.01, 0.1, 0.5, 1.0],
                         'l1_ratio': [0.1, 0.5, 0.9, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [144]:
y_pred_2 = grid_search.predict(X_test_encoded)
display(display(pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_2})))

,Actual,Predicted
1388,559.0,1539.179735
50052,2201.0,2396.887566
41645,1238.0,2016.805121
42377,1304.0,1837.779895
17244,6901.0,11355.110803
...,...,...
44081,1554.0,2032.783004
23713,633.0,954.145010
31375,761.0,1549.000419
21772,9836.0,7724.390147


None

In [145]:
print("Best parameters from grid search:", grid_search.best_params_)
print("Accuracy 2:", grid_search.score(X_train_encoded, y_train))
print("Mean Squared Error 2:", mean_squared_error(y_test, y_pred_2))
print("R^2 Score 2:", r2_score(y_test, y_pred_2))
print("Mean Absolute Error 2:", mean_absolute_error(y_test, y_pred_2))
print("Root Mean Squared Error 2:", root_mean_squared_error(y_test, y_pred_2))

Best parameters from grid search: {'alpha': 1.0, 'l1_ratio': 0.9}
Accuracy 2: -1464771.2593357505
Mean Squared Error 2: 1499314.3514900785
R^2 Score 2: 0.9056846454439307
Mean Absolute Error 2: 755.2902511088514
Root Mean Squared Error 2: 1224.464924565044
